### 命令模式 

想知道如何在应用中实现撤销功能。你已读过本章的标题，所以知道
应该推荐哪个设计模式来实现撤销，那就是命令模式（Command pattern）。         
命令设计模式帮助我们将一个操作（撤销、重做、复制、粘贴等）封装成一个对象。简而言
之，这意味着创建一个类，包含实现该操作所需要的所有逻辑和方法。这样做的优势如下所述 （请
 网页［t.cn/Rqr3tfQ］）。    
 我们并不需要直接执行一个命令。命令可以按照希望执行。    
 调用命令的对象与知道如何执行命令的对象解耦。调用者无需知道命令的任何实现细节。       
 如果有意义，可以把多个命令组织起来，这样调用者能够按顺序执行它们。例如，在实
现一个多层撤销命令时，这是很有用的。    

### 现实生活的例子 
当我们去餐馆吃饭时，会叫服务员来点单。他们用来做记录的账单（通常是纸质的）就是
命令模式的一个例子。在记录好订单后，服务员将其放入账单队列，厨师会照着单子去做。每
个账单都是独立的，并且可用来执行许多不同命令，例如，一个命令对应一个将要烹饪的菜品。

### 应用案例    
许多开发人员以为撤销例子是命令模式的唯一应用案例。撤销操作确实是命令模式的杀手级
特性，然而命令模式能做的实际上还有很多（请参考［GOF95，第265页］和网页［t.cn/R4a50r2］）。     
 GUI按钮和菜单项：前面提过的PyQt例子使用命令模式来实现按钮和菜单项上的动作。    
 其他操作：除了撤销，命令模式可用于实现任何操作。其中一些例子包括剪切、复制、
粘贴、重做和文本大写。       
 事务型行为和日志记录：事务型行为和日志记录对于为变更记录一份持久化日志是很重
要的。操作系统用它来从系统崩溃中恢复，关系型数据库用它来实现事务，文件系统用
它来实现快照，而安装程序（向导程序）用它来恢复取消的安装。      
 宏：在这里，宏是指一个动作序列，可在任意时间点按要求进行录制和执行。流行的编
辑器（比如，Emacs和Vim）都支持宏   

### 实现 
本节中，我们将使用命令模式实现最基本的文件操作工具。    
 创建一个文件，并随意写入一个字符串   
 读取一个文件的内容    
 重命名一个文件    
 删除一个文件    
我们并不从头实现这些工具程序，因为Python在os模块中已提供了良好的实现。我们想做的
是在已有实现之上添加一个额外的抽象层，这样可以当作命令来使用。这样，我们就能获得命令
提供的所有优势。 

每个命令都包括两个部分，初始化部分和执行部分。初始化部分由__init__()方法完成，
包含该命令发挥作用所要求的所有信息（文件路径和将写入文件的内容等） 。执行部分由
execute()方法完成。在我们想真正地运行命令时才调用其execute()方法。该方法并不需要
在命令初始化之后立即调用。 
我们从重命名工具开始，使用RenameFile类来实现。__init__()方法接受源文件路径（path_src）和目标文件路径（path_dest）作为参数。如果文件路径未使用路径分隔符，则
在当前目录下创建文件。使用路径分隔符的一个例子是传递字符串/tmp/file1作为path_src，
字符串/home/user/file2作为path_dest。不使用路径的例子则是传递file1作为path_src，
file2作为path_dest。 

In [3]:
import os

verbose = True


class RenameFile:

    def __init__(self, path_src, path_dest):
        self.src, self.dest = path_src, path_dest

    def execute(self):
        if verbose:
            print("[renaming '{}' to '{}']".format(self.src, self.dest))
        os.rename(self.src, self.dest)

    def undo(self):
        if verbose:
            print("[renaming '{}' back to '{}']".format(self.dest, self.src))
        os.rename(self.dest, self.src)


class CreateFile:

    def __init__(self, path, txt='hello world\n'):
        self.path, self.txt = path, txt

    def execute(self):
        if verbose:
            print("[creating file '{}']".format(self.path))
        with open(self.path, mode='w', encoding='utf-8') as out_file:
            out_file.write(self.txt)

    def undo(self):
        delete_file(self.path)


class ReadFile:

    def __init__(self, path):
        self.path = path

    def execute(self):
        if verbose:
            print("[reading file '{}']".format(self.path))
        with open(self.path, mode='r', encoding='utf-8') as in_file:
            print(in_file.read(), end='')


def delete_file(path):
    if verbose:
        print("deleting file '{}'".format(path))
    os.remove(path)


def main():
    orig_name, new_name = 'file1', 'file2'

    commands = []
    for cmd in CreateFile(orig_name), ReadFile(orig_name), RenameFile(orig_name, new_name):
        commands.append(cmd)

    [c.execute() for c in commands]

    answer = input('reverse the executed commands? [y/n] ')

    if answer not in 'yY':
        print("the result is {}".format(new_name))
        exit()

    for c in reversed(commands):
        try:
            c.undo()
        except AttributeError as e:
            pass


In [4]:
main()

[creating file 'file1']
[reading file 'file1']
hello world
[renaming 'file1' to 'file2']
reverse the executed commands? [y/n] y
[renaming 'file2' back to 'file1']
deleting file 'file1'


### 软件的例子 
PyQt是QT工具包的Python绑定。PyQt包含一个QAction类，将一个动作建模为一个命令。
对每个动作都支持额外的可选信息，比如，描述、工具提示、快捷键和其他（请参考网页
［t.cn/Rqr3VQU］）。 
git-cola（请参考网页［t.cn/Rqr3IWK］）是使用Python语言编写的一个Git GUI，它使用命令
模式来修改模型、变更一次提交、应用一个差异选择、签出，等等（请参考网页［t.cn/Rqr3JVz］）。